In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import requests
import torch
import torch.nn.functional as F
import torchtext
import torch.nn as nn
import random
import tarfile

In [ ]:
! pip install datasets transformers sacrebleu

In [ ]:
!pip install accelerate -U
!pip install sentencepiece

  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (9

In [ ]:
!pip install xformers
!pip install optimum
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch] -U

In [ ]:
import gc

def free_gpu():
  torch.cuda.empty_cache()
  gc.collect()

In [ ]:
from torch import cuda
device = 'cuda'if cuda.is_available() else 'cpu'
!nvidia-smi

Sun Jun 23 05:21:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
print(torch.__version__)
print(torch.version.cuda)

2.3.0+cu121
12.1


In [ ]:
import torch, os
import numpy as np
import random

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
# Constants
# MODELNAME ='iwslt15-en-vi-bilstm.model'
# EPOCH = 110
# BATCHSIZE = 32
# LR = 0.0001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Download and extract data
def iwslt15(train_test):
    url ='https://github.com/stefan-it/nmt-en-vi/raw/master/data/'
    r = requests.get(url + train_test + '-en-vi.tgz')
    filename = train_test + '-en-vi.tgz'
    with open(filename, 'wb') as f:
        f.write(r.content)
        tarfile.open(filename, "r:gz").extractall("iwslt15")

# Load data
iwslt15('train')
iwslt15('test-2013')

In [ ]:
import os

# List all files in the current directory
files = os.listdir()

# Print the list of files
for file in files:
    print(file)

.config
test-2013-en-vi.tgz
train-en-vi.tgz
iwslt15
gdrive
sample_data


In [ ]:
import tarfile
import os

# Open the .tgz file
tar = tarfile.open('train-en-vi.tgz', 'r:gz')

# Extract all files into the current directory
tar.extractall()

# Close the .tgz file
tar.close()

In [ ]:
!pip install bpemb

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer

# Đọc file 'train.en' vào DataFrame
df_train_en = pd.read_csv('train.en', delimiter="\t", header=None)
rtpe =[121099,121582]
print("Before replacement:")
values_before_replacement = []
for row in rtpe:
    if row < len(df_train_en):
        values_before_replacement.append((row, df_train_en.iloc[row, 0]))
        print(f"Row {row}: {df_train_en.iloc[row, 0]}")
# Thay thế các giá trị không hợp lệ bằng NaN trong DataFrame 'df_train_vi'
for row in rtpe:
    if row < len(df_train_en):
        df_train_en.at[row, 0] = np.nan

df_train_en.dropna(inplace=True)
# Đọc file 'train.vi' vào DataFrame
df_train_vi = pd.read_csv('train.vi', delimiter="\t", header=None)

# Danh sách các dòng cần thay thế bằng NaN, đã trừ đi 2 để đúng với DataFrame
rows_to_replace = [469, 8695, 9762, 10707, 21738, 26408, 29494, 38600, 41017, 48825, 50894, 51586,
                   54158, 56297, 57140, 57746, 58330, 66260, 68755, 73578, 76896, 77188, 79669,
                   87612, 88637, 89931, 94060, 104125, 104677, 107006, 112823, 115182, 116898,
                   123074, 124916, 125209, 128025,130121, 130770]
# In ra các giá trị của các ô trước khi thay thế
print("Before replacement:")
values_before_replacement = []
for row in rows_to_replace:
    if row < len(df_train_vi):
        values_before_replacement.append((row, df_train_vi.iloc[row, 0]))
        print(f"Row {row}: {df_train_vi.iloc[row, 0]}")

# Thay thế các giá trị không hợp lệ bằng NaN trong DataFrame 'df_train_vi'
for row in rows_to_replace:
    if row < len(df_train_vi):
        df_train_vi.at[row, 0] = np.nan

# In ra các giá trị đã thay thế để kiểm tra xem các giá trị có đúng là NaN không
print("\nReplaced values (should be NaN):")
for row in rows_to_replace:
    if row < len(df_train_vi):
        print(f"Row {row}: {df_train_vi.iloc[row, 0]}")
df_train_vi.dropna(inplace=True)

# Đổi tên các cột
df_train_en.columns = ['en']
df_train_vi.columns = ['vi']

# Nối hai DataFrame lại với nhau
df_train = pd.concat([df_train_vi.reset_index(drop=True), df_train_en.reset_index(drop=True)], axis=1)

# Chuyển cột 'en' sang kiểu string
df_train['en'] = df_train['en'].astype(str)


Before replacement:
Row 121099: .
Row 121582: .
Before replacement:
Row 469: ̣
Row 8695: Khán giả vỗ tay
Row 9762: Vỗ tay .
Row 10707: vỗ tay
Row 21738: Vỗ tay
Row 26408: vỗ tay
Row 29494: Vỗ tay .
Row 38600: &quot; tán thưởng &quot;
Row 41017: Vỗ tay
Row 48825: Vỗ tay
Row 50894: .
Row 51586: .
Row 54158: Vỗ tay
Row 56297: .
Row 57140: Vỗ tay
Row 57746: .
Row 58330: Vỗ tay
Row 66260: ̣
Row 68755: Cười
Row 73578: Vỗ tay
Row 76896: Vỗ tay
Row 77188: .
Row 79669: .
Row 87612: .
Row 88637: .
Row 89931: Vỗ tay
Row 94060: Vỗ tay .
Row 104125: Vỗ tay
Row 104677: Vỗ tay
Row 107006: Vỗ tay
Row 112823: vỗ tay
Row 115182: Vỗ tay
Row 116898: Vỗ tay .
Row 123074: Cám ơn
Row 124916: .
Row 125209: .
Row 128025: .
Row 130121: có những giấc mơ đẹp .
Row 130770: .

Replaced values (should be NaN):
Row 469: nan
Row 8695: nan
Row 9762: nan
Row 10707: nan
Row 21738: nan
Row 26408: nan
Row 29494: nan
Row 38600: nan
Row 41017: nan
Row 48825: nan
Row 50894: nan
Row 51586: nan
Row 54158: nan
Row 56297: nan
Row

In [ ]:
from bpemb import BPEmb

bpemb_vi = BPEmb(lang="vi", vs=50000, dim=200)
bpemb_en = BPEmb(lang="en", vs=50000, dim=200)

downloading https://nlp.h-its.org/bpemb/vi/vi.wiki.bpe.vs50000.model


100%|██████████| 1041864/1041864 [00:00<00:00, 1392797.67B/s]


downloading https://nlp.h-its.org/bpemb/vi/vi.wiki.bpe.vs50000.d200.w2v.bin.tar.gz


100%|██████████| 37483223/37483223 [00:02<00:00, 14283996.14B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs50000.model


100%|██████████| 1100587/1100587 [00:00<00:00, 1480697.21B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs50000.d200.w2v.bin.tar.gz


100%|██████████| 37527847/37527847 [00:03<00:00, 12259737.49B/s]


In [ ]:
def tokenizer(text, lang):
  if lang == "vi":
    text_pp = " ".join(bpemb_vi.encode(text))
  else:
    text_pp = " ".join(bpemb_en.encode(text))

  return text_pp

In [ ]:
def token_len(text):
    return len(text.replace("▁", "").split(" "))

In [ ]:
df_train["vi_token"] = df_train["vi"].apply(lambda x: tokenizer(x, "vi"))
df_train["en_token"] = df_train["en"].apply(lambda x: tokenizer(x, "en"))

# In ra số lượng giá trị không phải là NaN trong mỗi cột
print("\nCount of non-NaN values in each column:")
print(df_train.count())



Count of non-NaN values in each column:
vi          133166
en          133166
vi_token    133166
en_token    133166
dtype: int64


In [ ]:
tokenizer(df_train["vi"][0], "vi")

'▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ▁hậu'

In [ ]:
df_train = df_train.head(133001)
df_train

,vi,en,vi_token,en_token
0,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike : The science behind a climate hea...,▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ...,▁rachel ▁pike ▁: ▁the ▁science ▁behind ▁a ▁cli...
1,"Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike...","▁trong ▁0 ▁phút ▁, ▁chuyên ▁gia ▁hoá ▁học ▁khí...","▁in ▁0 ▁minutes ▁, ▁atmospheric ▁chemist ▁rach..."
2,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...,▁tôi ▁muốn ▁cho ▁các ▁bạn ▁biết ▁về ▁sự ▁to ▁l...,▁i ▁& ap os ; d ▁like ▁to ▁talk ▁to ▁you ▁toda...
3,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...,▁có ▁những ▁dòng ▁trông ▁như ▁thế ▁này ▁khi ▁b...,▁headlines ▁that ▁look ▁like ▁this ▁when ▁they...
4,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...,▁cả ▁hai ▁đều ▁là ▁một ▁nhánh ▁của ▁cùng ▁một ...,▁they ▁are ▁both ▁two ▁branches ▁of ▁the ▁same...
...,...,...,...,...
132996,"Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was...",▁mà ▁chính ▁hệ ▁thống ▁y ▁tế ▁mà ▁tôi ▁góp ▁ph...,"▁maybe ▁the ▁system ▁, ▁of ▁which ▁i ▁was ▁a ▁..."
132997,"Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you...",▁nếu ▁giờ ▁đây ▁cô ▁đang ▁xem ▁chương ▁trình ▁...,"▁if ▁you ▁& ap os ; re ▁watching ▁this ▁now ▁,..."
132998,Paul Pholeros : Làm sao để bớt nghèo khổ ? Hãy...,Paul Pholeros : How to reduce poverty ? Fix homes,▁paul ▁ph ol eros ▁: ▁làm ▁sao ▁để ▁bớt ▁nghèo...,▁paul ▁ph ol eros ▁: ▁how ▁to ▁reduce ▁poverty...
132999,"Năm 1985 , kiến trúc sư Paul Pholeros được gia...","In 1985 , architect Paul Pholeros was challeng...","▁năm ▁0000 ▁, ▁kiến ▁trúc ▁sư ▁paul ▁ph ol ero...","▁in ▁0000 ▁, ▁architect ▁paul ▁ph ol eros ▁was..."


In [ ]:
import tarfile
import os

# Open the .tgz file
tar = tarfile.open('test-2013-en-vi.tgz', 'r:gz')

# Extract all files into the current directory
tar.extractall()

# Close the .tgz file
tar.close()


In [ ]:
import pandas as pd
df_test_en = pd.read_csv('tst2013.en', delimiter = "\t", header=None)

df_test_vi = pd.read_csv('tst2013.vi', delimiter = "\t", header=None)

import pandas as pd
import re

# Rename the columns
df_test_en.columns = ['en']
df_test_vi.columns = ['vi']

# Concatenate the dataframes
df_test = pd.concat([df_test_vi, df_test_en], axis=1)
# Convert 'en' column to string and then create the 'en_token' column
df_test['en'] = df_test['en'].astype(str)


df_test["vi_token"] = df_test["vi"].apply(lambda x: tokenizer(x, "vi"))
df_test["en_token"] = df_test["en"].apply(lambda x: tokenizer(x, "en"))

In [ ]:
df_test

,vi,en,vi_token,en_token
0,"Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t...","▁khi ▁tôi ▁còn ▁nhỏ ▁, ▁tôi ▁nghĩ ▁rằng ▁bắc t...","▁when ▁i ▁was ▁little ▁, ▁i ▁thought ▁my ▁coun..."
1,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .,▁tôi ▁đã ▁rất ▁tự ▁hào ▁về ▁đất ▁nước ▁tôi ▁.,▁and ▁i ▁was ▁very ▁proud ▁.
2,"Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th...","▁ở ▁trường ▁, ▁chúng ▁tôi ▁dành ▁rất ▁nhiều ▁t...","▁in ▁school ▁, ▁we ▁spent ▁a ▁lot ▁of ▁time ▁s..."
3,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...,▁mặc ▁dù ▁tôi ▁đã ▁từng ▁tự ▁hỏi ▁không ▁biết ...,▁although ▁i ▁often ▁wondered ▁about ▁the ▁out...
4,"Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu...","▁khi ▁tôi ▁lên ▁0 ▁, ▁tôi ▁chứng ▁kiến ▁cảnh ▁...","▁when ▁i ▁was ▁seven ▁years ▁old ▁, ▁i ▁saw ▁m..."
...,...,...,...,...
1263,"Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as...","▁tôi ▁thực ▁sự ▁tin ▁, ▁nếu ▁ta ▁coi ▁người ▁k...","▁i ▁truly ▁believe ▁, ▁if ▁we ▁can ▁see ▁one ▁..."
1264,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...,▁những ▁tấm ▁hình ▁không ▁phải ▁là ▁về ▁bản ▁t...,▁these ▁images ▁are ▁not ▁of ▁issues ▁. ▁they ...
1265,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...,▁không ▁có ▁ngày ▁nào ▁mà ▁tôi ▁không ▁nghĩ ▁v...,▁there ▁is ▁not ▁a ▁day ▁that ▁goes ▁by ▁that ...
1266,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...,▁tôi ▁hi ▁vọng ▁những ▁tấm ▁hình ▁sẽ ▁đánh ▁th...,▁i ▁hope ▁that ▁these ▁images ▁awaken ▁a ▁forc...


In [ ]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 8.7 MB/s eta 0:00:00


In [ ]:
from sacremoses import MosesPunctNormalizer

mpn_vi = MosesPunctNormalizer(lang = "vi")
mpn_en = MosesPunctNormalizer(lang = "en")

def punc_norm(text, lang):
  if lang == "vi":
    return mpn_vi.normalize(text)

  return mpn_en.normalize(text)

In [ ]:
df_train["vi_punc"] = df_train["vi"].apply(lambda x: punc_norm(x, "vi"))
df_train["en_punc"] = df_train["en"].apply(lambda x: punc_norm(x, "en"))

df_train

,vi,en,vi_token,en_token,vi_punc,en_punc
0,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike : The science behind a climate hea...,▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ...,▁rachel ▁pike ▁: ▁the ▁science ▁behind ▁a ▁cli...,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike: The science behind a climate head...
1,"Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike...","▁trong ▁0 ▁phút ▁, ▁chuyên ▁gia ▁hoá ▁học ▁khí...","▁in ▁0 ▁minutes ▁, ▁atmospheric ▁chemist ▁rach...","Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike..."
2,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...,▁tôi ▁muốn ▁cho ▁các ▁bạn ▁biết ▁về ▁sự ▁to ▁l...,▁i ▁& ap os ; d ▁like ▁to ▁talk ▁to ▁you ▁toda...,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...
3,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...,▁có ▁những ▁dòng ▁trông ▁như ▁thế ▁này ▁khi ▁b...,▁headlines ▁that ▁look ▁like ▁this ▁when ▁they...,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...
4,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...,▁cả ▁hai ▁đều ▁là ▁một ▁nhánh ▁của ▁cùng ▁một ...,▁they ▁are ▁both ▁two ▁branches ▁of ▁the ▁same...,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...
...,...,...,...,...,...,...
132996,"Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was...",▁mà ▁chính ▁hệ ▁thống ▁y ▁tế ▁mà ▁tôi ▁góp ▁ph...,"▁maybe ▁the ▁system ▁, ▁of ▁which ▁i ▁was ▁a ▁...","Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was..."
132997,"Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you...",▁nếu ▁giờ ▁đây ▁cô ▁đang ▁xem ▁chương ▁trình ▁...,"▁if ▁you ▁& ap os ; re ▁watching ▁this ▁now ▁,...","Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you..."
132998,Paul Pholeros : Làm sao để bớt nghèo khổ ? Hãy...,Paul Pholeros : How to reduce poverty ? Fix homes,▁paul ▁ph ol eros ▁: ▁làm ▁sao ▁để ▁bớt ▁nghèo...,▁paul ▁ph ol eros ▁: ▁how ▁to ▁reduce ▁poverty...,Paul Pholeros: Làm sao để bớt nghèo khổ ? Hãy ...,Paul Pholeros: How to reduce poverty ? Fix homes
132999,"Năm 1985 , kiến trúc sư Paul Pholeros được gia...","In 1985 , architect Paul Pholeros was challeng...","▁năm ▁0000 ▁, ▁kiến ▁trúc ▁sư ▁paul ▁ph ol ero...","▁in ▁0000 ▁, ▁architect ▁paul ▁ph ol eros ▁was...","Năm 1985 , kiến trúc sư Paul Pholeros được gia...","In 1985 , architect Paul Pholeros was challeng..."


In [ ]:
df_test["vi_punc"] = df_test["vi"].apply(lambda x: punc_norm(x, "vi"))
df_test["en_punc"] = df_test["en"].apply(lambda x: punc_norm(x, "en"))

df_test

,vi,en,vi_token,en_token,vi_punc,en_punc
0,"Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t...","▁khi ▁tôi ▁còn ▁nhỏ ▁, ▁tôi ▁nghĩ ▁rằng ▁bắc t...","▁when ▁i ▁was ▁little ▁, ▁i ▁thought ▁my ▁coun...","Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t..."
1,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .,▁tôi ▁đã ▁rất ▁tự ▁hào ▁về ▁đất ▁nước ▁tôi ▁.,▁and ▁i ▁was ▁very ▁proud ▁.,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .
2,"Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th...","▁ở ▁trường ▁, ▁chúng ▁tôi ▁dành ▁rất ▁nhiều ▁t...","▁in ▁school ▁, ▁we ▁spent ▁a ▁lot ▁of ▁time ▁s...","Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th..."
3,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...,▁mặc ▁dù ▁tôi ▁đã ▁từng ▁tự ▁hỏi ▁không ▁biết ...,▁although ▁i ▁often ▁wondered ▁about ▁the ▁out...,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...
4,"Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu...","▁khi ▁tôi ▁lên ▁0 ▁, ▁tôi ▁chứng ▁kiến ▁cảnh ▁...","▁when ▁i ▁was ▁seven ▁years ▁old ▁, ▁i ▁saw ▁m...","Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu..."
...,...,...,...,...,...,...
1263,"Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as...","▁tôi ▁thực ▁sự ▁tin ▁, ▁nếu ▁ta ▁coi ▁người ▁k...","▁i ▁truly ▁believe ▁, ▁if ▁we ▁can ▁see ▁one ▁...","Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as..."
1264,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...,▁những ▁tấm ▁hình ▁không ▁phải ▁là ▁về ▁bản ▁t...,▁these ▁images ▁are ▁not ▁of ▁issues ▁. ▁they ...,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...
1265,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...,▁không ▁có ▁ngày ▁nào ▁mà ▁tôi ▁không ▁nghĩ ▁v...,▁there ▁is ▁not ▁a ▁day ▁that ▁goes ▁by ▁that ...,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...
1266,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...,▁tôi ▁hi ▁vọng ▁những ▁tấm ▁hình ▁sẽ ▁đánh ▁th...,▁i ▁hope ▁that ▁these ▁images ▁awaken ▁a ▁forc...,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...


In [ ]:
df_train["vi_token"] = df_train["vi_punc"].apply(lambda x: tokenizer(x, "vi"))
df_train["en_token"] = df_train["en_punc"].apply(lambda x: tokenizer(x, "en"))

df_train

,vi,en,vi_token,en_token,vi_punc,en_punc
0,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike : The science behind a climate hea...,▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ...,▁rachel ▁pike : ▁the ▁science ▁behind ▁a ▁clim...,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike: The science behind a climate head...
1,"Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike...","▁trong ▁0 ▁phút ▁, ▁chuyên ▁gia ▁hoá ▁học ▁khí...","▁in ▁0 ▁minutes ▁, ▁atmospheric ▁chemist ▁rach...","Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike..."
2,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...,▁tôi ▁muốn ▁cho ▁các ▁bạn ▁biết ▁về ▁sự ▁to ▁l...,▁i ▁& ap os ; d ▁like ▁to ▁talk ▁to ▁you ▁toda...,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...
3,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...,▁có ▁những ▁dòng ▁trông ▁như ▁thế ▁này ▁khi ▁b...,▁headlines ▁that ▁look ▁like ▁this ▁when ▁they...,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...
4,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...,▁cả ▁hai ▁đều ▁là ▁một ▁nhánh ▁của ▁cùng ▁một ...,▁they ▁are ▁both ▁two ▁branches ▁of ▁the ▁same...,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...
...,...,...,...,...,...,...
132996,"Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was...",▁mà ▁chính ▁hệ ▁thống ▁y ▁tế ▁mà ▁tôi ▁góp ▁ph...,"▁maybe ▁the ▁system ▁, ▁of ▁which ▁i ▁was ▁a ▁...","Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was..."
132997,"Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you...",▁nếu ▁giờ ▁đây ▁cô ▁đang ▁xem ▁chương ▁trình ▁...,"▁if ▁you ▁& ap os ; re ▁watching ▁this ▁now ▁,...","Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you..."
132998,Paul Pholeros : Làm sao để bớt nghèo khổ ? Hãy...,Paul Pholeros : How to reduce poverty ? Fix homes,▁paul ▁ph ol eros : ▁làm ▁sao ▁để ▁bớt ▁nghèo ...,▁paul ▁ph ol eros : ▁how ▁to ▁reduce ▁poverty ...,Paul Pholeros: Làm sao để bớt nghèo khổ ? Hãy ...,Paul Pholeros: How to reduce poverty ? Fix homes
132999,"Năm 1985 , kiến trúc sư Paul Pholeros được gia...","In 1985 , architect Paul Pholeros was challeng...","▁năm ▁0000 ▁, ▁kiến ▁trúc ▁sư ▁paul ▁ph ol ero...","▁in ▁0000 ▁, ▁architect ▁paul ▁ph ol eros ▁was...","Năm 1985 , kiến trúc sư Paul Pholeros được gia...","In 1985 , architect Paul Pholeros was challeng..."


In [ ]:
df_test["vi_token"] = df_test["vi_punc"].apply(lambda x: tokenizer(x, "vi"))
df_test["en_token"] = df_test["en_punc"].apply(lambda x: tokenizer(x, "en"))

df_test

,vi,en,vi_token,en_token,vi_punc,en_punc
0,"Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t...","▁khi ▁tôi ▁còn ▁nhỏ ▁, ▁tôi ▁nghĩ ▁rằng ▁bắc t...","▁when ▁i ▁was ▁little ▁, ▁i ▁thought ▁my ▁coun...","Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t..."
1,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .,▁tôi ▁đã ▁rất ▁tự ▁hào ▁về ▁đất ▁nước ▁tôi ▁.,▁and ▁i ▁was ▁very ▁proud ▁.,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .
2,"Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th...","▁ở ▁trường ▁, ▁chúng ▁tôi ▁dành ▁rất ▁nhiều ▁t...","▁in ▁school ▁, ▁we ▁spent ▁a ▁lot ▁of ▁time ▁s...","Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th..."
3,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...,▁mặc ▁dù ▁tôi ▁đã ▁từng ▁tự ▁hỏi ▁không ▁biết ...,▁although ▁i ▁often ▁wondered ▁about ▁the ▁out...,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...
4,"Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu...","▁khi ▁tôi ▁lên ▁0 ▁, ▁tôi ▁chứng ▁kiến ▁cảnh ▁...","▁when ▁i ▁was ▁seven ▁years ▁old ▁, ▁i ▁saw ▁m...","Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu..."
...,...,...,...,...,...,...
1263,"Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as...","▁tôi ▁thực ▁sự ▁tin ▁, ▁nếu ▁ta ▁coi ▁người ▁k...","▁i ▁truly ▁believe ▁, ▁if ▁we ▁can ▁see ▁one ▁...","Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as..."
1264,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...,▁những ▁tấm ▁hình ▁không ▁phải ▁là ▁về ▁bản ▁t...,▁these ▁images ▁are ▁not ▁of ▁issues ▁. ▁they ...,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...
1265,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...,▁không ▁có ▁ngày ▁nào ▁mà ▁tôi ▁không ▁nghĩ ▁v...,▁there ▁is ▁not ▁a ▁day ▁that ▁goes ▁by ▁that ...,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...
1266,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...,▁tôi ▁hi ▁vọng ▁những ▁tấm ▁hình ▁sẽ ▁đánh ▁th...,▁i ▁hope ▁that ▁these ▁images ▁awaken ▁a ▁forc...,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...


In [ ]:
data_vi = df_train["vi_token"].values.tolist()
data_en = df_train["en_token"].values.tolist()

with open("train.vi", "w") as f:
  f.write("\n".join(data_vi))

with open("train.en", "w") as f:
  f.write("\n".join(data_en))

In [ ]:
!wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/training/clean-corpus-n.perl

--2024-06-23 05:23:35--  https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/training/clean-corpus-n.perl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4257 (4.2K) [text/plain]
Saving to: ‘clean-corpus-n.perl’

clean-corpus-n.perl 100%[===================>]   4.16K  --.-KB/s    in 0s      

2024-06-23 05:23:35 (41.3 MB/s) - ‘clean-corpus-n.perl’ saved [4257/4257]



In [ ]:
!perl clean-corpus-n.perl -max-word-length 50 train vi en train.clean 0 512

clean-corpus.perl: processing train.vi & .en to train.clean, cutoff 0-512, ratio 9
..........(100000)...
Input sentences: 133001  Output sentences:  132994


In [ ]:
data_test_vi = df_test["vi_token"].values.tolist()
data_test_en = df_test["en_token"].values.tolist()

with open("test.vi", "w") as f:
  f.write("\n".join(data_test_vi))

with open("test.en", "w") as f:
  f.write("\n".join(data_test_en))

In [ ]:
!perl clean-corpus-n.perl -max-word-length 50 test vi en test.clean 0 512

clean-corpus.perl: processing test.vi & .en to test.clean, cutoff 0-512, ratio 9

Input sentences: 1268  Output sentences:  1268


In [ ]:
!wc -l train.clean.vi train.clean.en

  132994 train.clean.vi
  132994 train.clean.en
  265988 total


In [ ]:
!wc -l test.clean.vi test.clean.en

  1268 test.clean.vi
  1268 test.clean.en
  2536 total


In [ ]:
with open("train.clean.vi", "r") as f:
    data_train_vi = f.readlines()

with open("train.clean.en", "r") as f:
    data_train_en = f.readlines()

data_train_vi = [i.replace("\n", "") for i in data_train_vi]
data_train_en = [i.replace("\n", "") for i in data_train_en]
data_train_vi[:10]

['▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ▁hậu',
 '▁trong ▁0 ▁phút ▁, ▁chuyên ▁gia ▁hoá ▁học ▁khí ▁quyển ▁rachel ▁pike ▁giới ▁thiệu ▁sơ ▁lược ▁về ▁những ▁nỗ ▁lực ▁khoa ▁học ▁miệt ▁mài ▁đằng ▁sau ▁những ▁tiêu ▁đề ▁táo ▁bạo ▁về ▁biến ▁đổi ▁khí ▁hậu ▁, ▁cùng ▁với ▁đoàn ▁nghiên ▁cứu ▁của ▁mình ▁-- ▁hàng ▁ngàn ▁người ▁đã ▁cống ▁hiến ▁cho ▁dự ▁án ▁này ▁-- ▁một ▁chuyến ▁bay ▁mạo ▁hiểm ▁qua ▁rừng ▁già ▁để ▁tìm ▁kiếm ▁thông ▁tin ▁về ▁một ▁phân ▁tử ▁then ▁chốt ▁.',
 '▁tôi ▁muốn ▁cho ▁các ▁bạn ▁biết ▁về ▁sự ▁to ▁lớn ▁của ▁những ▁nỗ ▁lực ▁khoa ▁học ▁đã ▁góp ▁phần ▁làm ▁nên ▁các ▁dòng ▁tít ▁bạn ▁thường ▁thấy ▁trên ▁báo ▁.',
 '▁có ▁những ▁dòng ▁trông ▁như ▁thế ▁này ▁khi ▁bàn ▁về ▁biến ▁đổi ▁khí ▁hậu ▁, ▁và ▁như ▁thế ▁này ▁khi ▁nói ▁về ▁chất ▁lượng ▁không ▁khí ▁hay ▁khói ▁bụi ▁.',
 '▁cả ▁hai ▁đều ▁là ▁một ▁nhánh ▁của ▁cùng ▁một ▁lĩnh ▁vực ▁trong ▁ngành ▁khoa ▁học ▁khí ▁quyển ▁.',
 '▁các ▁tiêu ▁đề ▁gần ▁đây ▁trông ▁như ▁thế ▁này ▁khi ▁ban ▁điều ▁hành ▁biến ▁đổi ▁khí ▁hậu ▁liên ▁chính ▁phủ ▁, ▁gọ

In [ ]:
df_train = pd.DataFrame({"vi": data_train_vi, "en": data_train_en})
df_train

,vi,en
0,▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ...,▁rachel ▁pike : ▁the ▁science ▁behind ▁a ▁clim...
1,"▁trong ▁0 ▁phút ▁, ▁chuyên ▁gia ▁hoá ▁học ▁khí...","▁in ▁0 ▁minutes ▁, ▁atmospheric ▁chemist ▁rach..."
2,▁tôi ▁muốn ▁cho ▁các ▁bạn ▁biết ▁về ▁sự ▁to ▁l...,▁i ▁& ap os ; d ▁like ▁to ▁talk ▁to ▁you ▁toda...
3,▁có ▁những ▁dòng ▁trông ▁như ▁thế ▁này ▁khi ▁b...,▁headlines ▁that ▁look ▁like ▁this ▁when ▁they...
4,▁cả ▁hai ▁đều ▁là ▁một ▁nhánh ▁của ▁cùng ▁một ...,▁they ▁are ▁both ▁two ▁branches ▁of ▁the ▁same...
...,...,...
132989,▁mà ▁chính ▁hệ ▁thống ▁y ▁tế ▁mà ▁tôi ▁góp ▁ph...,"▁maybe ▁the ▁system ▁, ▁of ▁which ▁i ▁was ▁a ▁..."
132990,▁nếu ▁giờ ▁đây ▁cô ▁đang ▁xem ▁chương ▁trình ▁...,"▁if ▁you ▁& ap os ; re ▁watching ▁this ▁now ▁,..."
132991,▁paul ▁ph ol eros : ▁làm ▁sao ▁để ▁bớt ▁nghèo ...,▁paul ▁ph ol eros : ▁how ▁to ▁reduce ▁poverty ...
132992,"▁năm ▁0000 ▁, ▁kiến ▁trúc ▁sư ▁paul ▁ph ol ero...","▁in ▁0000 ▁, ▁architect ▁paul ▁ph ol eros ▁was..."


In [ ]:
df_test = pd.DataFrame({"vi": data_test_vi, "en": data_test_en})
df_test

,vi,en
0,"▁khi ▁tôi ▁còn ▁nhỏ ▁, ▁tôi ▁nghĩ ▁rằng ▁bắc t...","▁when ▁i ▁was ▁little ▁, ▁i ▁thought ▁my ▁coun..."
1,▁tôi ▁đã ▁rất ▁tự ▁hào ▁về ▁đất ▁nước ▁tôi ▁.,▁and ▁i ▁was ▁very ▁proud ▁.
2,"▁ở ▁trường ▁, ▁chúng ▁tôi ▁dành ▁rất ▁nhiều ▁t...","▁in ▁school ▁, ▁we ▁spent ▁a ▁lot ▁of ▁time ▁s..."
3,▁mặc ▁dù ▁tôi ▁đã ▁từng ▁tự ▁hỏi ▁không ▁biết ...,▁although ▁i ▁often ▁wondered ▁about ▁the ▁out...
4,"▁khi ▁tôi ▁lên ▁0 ▁, ▁tôi ▁chứng ▁kiến ▁cảnh ▁...","▁when ▁i ▁was ▁seven ▁years ▁old ▁, ▁i ▁saw ▁m..."
...,...,...
1263,"▁tôi ▁thực ▁sự ▁tin ▁, ▁nếu ▁ta ▁coi ▁người ▁k...","▁i ▁truly ▁believe ▁, ▁if ▁we ▁can ▁see ▁one ▁..."
1264,▁những ▁tấm ▁hình ▁không ▁phải ▁là ▁về ▁bản ▁t...,▁these ▁images ▁are ▁not ▁of ▁issues ▁. ▁they ...
1265,▁không ▁có ▁ngày ▁nào ▁mà ▁tôi ▁không ▁nghĩ ▁v...,▁there ▁is ▁not ▁a ▁day ▁that ▁goes ▁by ▁that ...
1266,▁tôi ▁hi ▁vọng ▁những ▁tấm ▁hình ▁sẽ ▁đánh ▁th...,▁i ▁hope ▁that ▁these ▁images ▁awaken ▁a ▁forc...


In [ ]:
def remove_punc(text):
    return text.replace("▁", " ")

In [ ]:
# columns_to_drop = ['vi', 'en']
# df_train = df_train.drop(columns=columns_to_drop)
# df_train = df_train.rename(columns = {
#                                         'vi_token': 'vi',
#                                         'en_token': 'en'
#                                     })

# df_train

In [ ]:
# columns_to_drop = ['vi', 'en']
# df_test = df_test.drop(columns=columns_to_drop)
# df_test = df_test.rename(columns = {
#                                         'vi_token': 'vi',
#                                         'en_token': 'en'
#                                     })

# df_test

In [ ]:
df_train["vi"] = df_train["vi"].apply(remove_punc)
df_train["en"] = df_train["en"].apply(remove_punc)
df_train
df_test["vi"] = df_test["vi"].apply(remove_punc)
df_test["en"] = df_test["en"].apply(remove_punc)
df_test

,vi,en
0,"khi tôi còn nhỏ , tôi nghĩ rằng bắc t...","when i was little , i thought my coun..."
1,tôi đã rất tự hào về đất nước tôi .,and i was very proud .
2,"ở trường , chúng tôi dành rất nhiều t...","in school , we spent a lot of time s..."
3,mặc dù tôi đã từng tự hỏi không biết ...,although i often wondered about the out...
4,"khi tôi lên 0 , tôi chứng kiến cảnh ...","when i was seven years old , i saw m..."
...,...,...
1263,"tôi thực sự tin , nếu ta coi người k...","i truly believe , if we can see one ..."
1264,những tấm hình không phải là về bản t...,these images are not of issues . they ...
1265,không có ngày nào mà tôi không nghĩ v...,there is not a day that goes by that ...
1266,tôi hi vọng những tấm hình sẽ đánh th...,i hope that these images awaken a forc...


In [ ]:
# import string
# import re
# def preprocessing(df):
#   df["en"] = df["en"].apply(lambda ele: ele.translate(str.maketrans('', '', string.punctuation))) # Remove punctuation
#   df["vi"] = df["vi"].apply(lambda ele: ele.translate(str.maketrans('', '', string.punctuation)))
#   df["en"] = df["en"].apply(lambda ele: ele.lower()) # convert text to lowercase
#   df["vi"] = df["vi"].apply(lambda ele: ele.lower())
#   df["en"] = df["en"].apply(lambda ele: ele.strip())
#   df["vi"] = df["vi"].apply(lambda ele: ele.strip())
#   df["en"] = df["en"].apply(lambda ele: re.sub("\s+", " ", ele))
#   df["vi"] = df["vi"].apply(lambda ele: re.sub("\s+", " ", ele))

#   return df

# df_train = preprocessing(df_train)
# df_test = preprocessing(df_test)
# df_train.head()

In [ ]:
df_train['vi']   = df_train['vi'].astype(str)
df_train['en']  = df_train['en'].astype(str)

df_test['vi']   = df_test['vi'].astype(str)
df_test['en']  = df_test['en'].astype(str)

In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")
metric

<ipython-input-47-dbdd67d4ba40>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


The repository for sacrebleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sacrebleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

In [ ]:
language_codes = {
    'af': 'Afrikaans',
    'af-ZA': 'Afrikaans (South Africa)',
    'ar': 'Arabic',
    'ar-AE': 'Arabic (U.A.E.)',
    'ar-BH': 'Arabic (Bahrain)',
    'ar-DZ': 'Arabic (Algeria)',
    'ar-EG': 'Arabic (Egypt)',
    'ar-IQ': 'Arabic (Iraq)',
    'ar-JO': 'Arabic (Jordan)',
    'ar-KW': 'Arabic (Kuwait)',
    'ar-LB': 'Arabic (Lebanon)',
    'ar-LY': 'Arabic (Libya)',
    'ar-MA': 'Arabic (Morocco)',
    'ar-OM': 'Arabic (Oman)',
    'ar-QA': 'Arabic (Qatar)',
    'ar-SA': 'Arabic (Saudi Arabia)',
    'ar-SY': 'Arabic (Syria)',
    'ar-TN': 'Arabic (Tunisia)',
    'ar-YE': 'Arabic (Yemen)',
    'az': 'Azeri (Latin)',
    'az-AZ': 'Azeri (Latin) (Azerbaijan)',
    'az-AZ': 'Azeri (Cyrillic) (Azerbaijan)',
    'be': 'Belarusian',
    'be-BY': 'Belarusian (Belarus)',
    'bg': 'Bulgarian',
    'bg-BG': 'Bulgarian (Bulgaria)',
    'bs-BA': 'Bosnian (Bosnia and Herzegovina)',
    'ca': 'Catalan',
    'ca-ES': 'Catalan (Spain)',
    'cs': 'Czech',
    'cs-CZ': 'Czech (Czech Republic)',
    'cy': 'Welsh',
    'cy-GB': 'Welsh (United Kingdom)',
    'da': 'Danish',
    'da-DK': 'Danish (Denmark)',
    'de': 'German',
    'de-AT': 'German (Austria)',
    'de-CH': 'German (Switzerland)',
    'de-DE': 'German (Germany)',
    'de-LI': 'German (Liechtenstein)',
    'de-LU': 'German (Luxembourg)',
    'dv': 'Divehi',
    'dv-MV': 'Divehi (Maldives)',
    'el': 'Greek',
    'el-GR': 'Greek (Greece)',
    'en': 'English',
    'en-AU': 'English (Australia)',
    'en-BZ': 'English (Belize)',
    'en-CA': 'English (Canada)',
    'en-CB': 'English (Caribbean)',
    'en-GB': 'English (United Kingdom)',
    'en-IE': 'English (Ireland)',
    'en-JM': 'English (Jamaica)',
    'en-NZ': 'English (New Zealand)',
    'en-PH': 'English (Republic of the Philippines)',
    'en-TT': 'English (Trinidad and Tobago)',
    'en-US': 'English (United States)',
    'en-ZA': 'English (South Africa)',
    'en-ZW': 'English (Zimbabwe)',
    'eo': 'Esperanto',
    'es': 'Spanish',
    'es-AR': 'Spanish (Argentina)',
    'es-BO': 'Spanish (Bolivia)',
    'es-CL': 'Spanish (Chile)',
    'es-CO': 'Spanish (Colombia)',
    'es-CR': 'Spanish (Costa Rica)',
    'es-DO': 'Spanish (Dominican Republic)',
    'es-EC': 'Spanish (Ecuador)',
    'es-ES': 'Spanish (Castilian)',
    'es-ES': 'Spanish (Spain)',
    'es-GT': 'Spanish (Guatemala)',
    'es-HN': 'Spanish (Honduras)',
    'es-MX': 'Spanish (Mexico)',
    'es-NI': 'Spanish (Nicaragua)',
    'es-PA': 'Spanish (Panama)',
    'es-PE': 'Spanish (Peru)',
    'es-PR': 'Spanish (Puerto Rico)',
    'es-PY': 'Spanish (Paraguay)',
    'es-SV': 'Spanish (El Salvador)',
    'es-UY': 'Spanish (Uruguay)',
    'es-VE': 'Spanish (Venezuela)',
    'et': 'Estonian',
    'et-EE': 'Estonian (Estonia)',
    'eu': 'Basque',
    'eu-ES': 'Basque (Spain)',
    'fa': 'Farsi',
    'fa-IR': 'Farsi (Iran)',
    'fi': 'Finnish',
    'fi-FI': 'Finnish (Finland)',
    'fo': 'Faroese',
    'fo-FO': 'Faroese (Faroe Islands)',
    'fr': 'French',
    'fr-BE': 'French (Belgium)',
    'fr-CA': 'French (Canada)',
    'fr-CH': 'French (Switzerland)',
    'fr-FR': 'French (France)',
    'fr-LU': 'French (Luxembourg)',
    'fr-MC': 'French (Principality of Monaco)',
    'gl': 'Galician',
    'gl-ES': 'Galician (Spain)',
    'gu': 'Gujarati',
    'gu-IN': 'Gujarati (India)',
    'he': 'Hebrew',
    'he-IL': 'Hebrew (Israel)',
    'hi': 'Hindi',
    'hi-IN': 'Hindi (India)',
    'hr': 'Croatian',
    'hr-BA': 'Croatian (Bosnia and Herzegovina)',
    'hr-HR': 'Croatian (Croatia)',
    'hu': 'Hungarian',
    'hu-HU': 'Hungarian (Hungary)',
    'hy': 'Armenian',
    'hy-AM': 'Armenian (Armenia)',
    'id': 'Indonesian',
    'id-ID': 'Indonesian (Indonesia)',
    'is': 'Icelandic',
    'is-IS': 'Icelandic (Iceland)',
    'it': 'Italian',
    'it-CH': 'Italian (Switzerland)',
    'it-IT': 'Italian (Italy)',
    'ja': 'Japanese',
    'ja-JP': 'Japanese (Japan)',
    'ka': 'Georgian',
    'ka-GE': 'Georgian (Georgia)',
    'kk': 'Kazakh',
    'kk-KZ': 'Kazakh (Kazakhstan)',
    'kn': 'Kannada',
    'kn-IN': 'Kannada (India)',
    'ko': 'Korean',
    'ko-KR': 'Korean (Korea)',
    'kok': 'Konkani',
    'kok-IN': 'Konkani (India)',
    'ky': 'Kyrgyz',
    'ky-KG': 'Kyrgyz (Kyrgyzstan)',
    'lt': 'Lithuanian',
    'lt-LT': 'Lithuanian (Lithuania)',
    'lv': 'Latvian',
    'lv-LV': 'Latvian (Latvia)',
    'mi': 'Maori',
    'mi-NZ': 'Maori (New Zealand)',
    'mk': 'FYRO Macedonian',
    'mk-MK': 'FYRO Macedonian (Former Yugoslav Republic of Macedonia)',
    'mn': 'Mongolian',
    'mn-MN': 'Mongolian (Mongolia)',
    'mr': 'Marathi',
    'mr-IN': 'Marathi (India)',
    'ms': 'Malay',
    'ms-BN': 'Malay (Brunei Darussalam)',
    'ms-MY': 'Malay (Malaysia)',
    'mt': 'Maltese',
    'mt-MT': 'Maltese (Malta)',
    'nb': 'Norwegian (Bokmål)',
    'nb-NO': 'Norwegian (Bokmål) (Norway)',
    'nl': 'Dutch',
    'nl-BE': 'Dutch (Belgium)',
    'nl-NL': 'Dutch (Netherlands)',
    'nn-NO': 'Norwegian (Nynorsk) (Norway)',
    'ns': 'Northern Sotho',
    'ns-ZA': 'Northern Sotho (South Africa)',
    'pa': 'Punjabi',
    'pa-IN': 'Punjabi (India)',
    'pl': 'Polish',
    'pl-PL': 'Polish (Poland)',
    'ps': 'Pashto',
    'ps-AR': 'Pashto (Afghanistan)',
    'pt': 'Portuguese',
    'pt-BR': 'Portuguese (Brazil)',
    'pt-PT': 'Portuguese (Portugal)',
    'qu': 'Quechua',
    'qu-BO': 'Quechua (Bolivia)',
    'qu-EC': 'Quechua (Ecuador)',
    'qu-PE': 'Quechua (Peru)',
    'ro': 'Romanian',
    'ro-RO': 'Romanian (Romania)',
    'ru': 'Russian',
    'ru-RU': 'Russian (Russia)',
    'sa': 'Sanskrit',
    'sa-IN': 'Sanskrit (India)',
    'se': 'Sami (Northern)',
    'se-FI': 'Sami (Northern) (Finland)',
    'se-FI': 'Sami (Skolt) (Finland)',
    'se-FI': 'Sami (Inari) (Finland)',
    'se-NO': 'Sami (Northern) (Norway)',
    'se-NO': 'Sami (Lule) (Norway)',
    'se-NO': 'Sami (Southern) (Norway)',
    'se-SE': 'Sami (Northern) (Sweden)',
    'se-SE': 'Sami (Lule) (Sweden)',
    'se-SE': 'Sami (Southern) (Sweden)',
    'sk': 'Slovak',
    'sk-SK': 'Slovak (Slovakia)',
    'sl': 'Slovenian',
    'sl-SI': 'Slovenian (Slovenia)',
    'sq': 'Albanian',
    'sq-AL': 'Albanian (Albania)',
    'sr-BA': 'Serbian (Latin) (Bosnia and Herzegovina)',
    'sr-BA': 'Serbian (Cyrillic) (Bosnia and Herzegovina)',
    'sr-SP': 'Serbian (Latin) (Serbia and Montenegro)',
    'sr-SP': 'Serbian (Cyrillic) (Serbia and Montenegro)',
    'sv': 'Swedish',
    'sv-FI': 'Swedish (Finland)',
    'sv-SE': 'Swedish (Sweden)',
    'sw': 'Swahili',
    'sw-KE': 'Swahili (Kenya)',
    'syr': 'Syriac',
    'syr-SY': 'Syriac (Syria)',
    'ta': 'Tamil',
    'ta-IN': 'Tamil (India)',
    'te': 'Telugu',
    'te-IN': 'Telugu (India)',
    'th': 'Thai',
    'th-TH': 'Thai (Thailand)',
    'tl': 'Tagalog',
    'tl-PH': 'Tagalog (Philippines)',
    'tn': 'Tswana',
    'tn-ZA': 'Tswana (South Africa)',
    'tr': 'Turkish',
    'tr-TR': 'Turkish (Turkey)',
    'tt': 'Tatar',
    'tt-RU': 'Tatar (Russia)',
    'ts': 'Tsonga',
    'uk': 'Ukrainian',
    'uk-UA': 'Ukrainian (Ukraine)',
    'ur': 'Urdu',
    'ur-PK': 'Urdu (Islamic Republic of Pakistan)',
    'uz': 'Uzbek (Latin)',
    'uz-UZ': 'Uzbek (Latin) (Uzbekistan)',
    'uz-UZ': 'Uzbek (Cyrillic) (Uzbekistan)',
    'vi': 'Vietnamese',
    'vi-VN': 'Vietnamese (Viet Nam)',
    'xh': 'Xhosa',
    'xh-ZA': 'Xhosa (South Africa)',
    'zh': 'Chinese',
    'zh-CN': 'Chinese (S)',
    'zh-HK': 'Chinese (Hong Kong)',
    'zh-MO': 'Chinese (Macau)',
    'zh-SG': 'Chinese (Singapore)',
    'zh-TW': 'Chinese (T)',
    'zu': 'Zulu',
    'zu-ZA': 'Zulu (South Africa)'
}

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer


src_lang = "vi"
tgt_lang = "en"
max_length = 128

model_name = "facebook/mbart-large-50-many-to-many-mmt"

def get_model(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

  model_type = model.config.model_type


  if "mbart" in model_type:
      langs = "ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN"
      langs = langs.split(",")

      for lang in langs:
          if src_lang in lang:
              tokenizer.src_lang = lang

          if tgt_lang in lang:
              tokenizer.tgt_lang = lang
  elif "marian" in model_type:
    langs = "vi_VN,en_XX"
    langs = langs.split(",")

    for lang in langs:
        if src_lang in lang:
            tokenizer.source_lang = lang

        if tgt_lang in lang:
            tokenizer.target_lang = lang

    if "mul" in model_name:
        prefix = ">>" + tokenizer.target_lang + "<< "

  if "t5" in model_type:
    prefix = f"translate {language_codes.get(src_lang, src_lang)} to {language_codes.get(tgt_lang, tgt_lang)}: "
  else:
    prefix = ""

  print(f"MODE TYPE\t:\t{model_type}")

  if "mbart" in model_type:
      print(f"SRC_LANG \t:\t{tokenizer.src_lang}")
      print(f"TGT_LANG \t:\t{tokenizer.tgt_lang}")
  elif "marian" in model_type:
      print(f"SRC_LANG \t:\t{tokenizer.source_lang}")
      print(f"TGT_LANG \t:\t{tokenizer.target_lang}")

  if prefix:
      print(f"PREFIX   \t:\t{prefix[:-3]}")
  else:
      print(f"PREFIX   \t:\tNone")

  return model, tokenizer, model_type, prefix

In [ ]:
def preprocess_function(tokenizer, prefix, src, tgt):
    inputs = prefix + src
    targets = tgt
    return tokenizer(text = inputs, text_target = targets, max_length=max_length, truncation=True)

In [ ]:

from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__ (self, tokenizer, prefix, X_data, y_data):
        self.tokenizer = tokenizer
        self.X_data = X_data
        self.y_data = y_data
        self.prefix = prefix
    def __len__ (self):
        return len(self.X_data)

    def __getitem__(self, idx):
        inputs = self.prefix + self.X_data[idx]
        targets = self.y_data[idx]

        inputs = self.tokenizer(inputs, max_length=max_length, truncation=True, return_tensors="pt", padding="max_length",)

        labels = self.tokenizer(text_target = targets, max_length=max_length, truncation=True, return_tensors="pt", padding="max_length",)

        return {
            'input_ids' : inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': labels.input_ids.squeeze(),
        }


In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [ ]:
free_gpu()

from optimum.bettertransformer import BetterTransformer

model, tokenizer, model_type, prefix = get_model(model_name)
# model = BetterTransformer.transform(model)

from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")

max_train = -1
if max_train > 0:
    train_dataset = CustomDataset(tokenizer, prefix, df_train[src_lang][:max_train], df_train[tgt_lang][:max_train])
else:
    train_dataset = CustomDataset(tokenizer, prefix, df_train[src_lang], df_train[tgt_lang])

test_dataset  = CustomDataset(tokenizer, prefix, df_test[src_lang], df_test[tgt_lang])


epochs = 1
learning_rate = 1e-4
weight_decay = 0.01
logging_steps = 100
batch_size = 8
seed = 42

free_gpu()

model_name = model_name.replace("/", "_")
training_args = Seq2SeqTrainingArguments(
    output_dir = f'/content/gdrive/MyDrive/Machine_Translation/models/{model_type}',
    overwrite_output_dir = True,

    learning_rate = learning_rate,
    num_train_epochs = epochs,
    weight_decay = weight_decay,
    # optim = "adamw_torch",

    # evaluation_strategy= 'epoch',
    save_strategy = 'no',
    # save_steps = logging_steps * 50,
    # eval_steps = logging_steps * 10,

    # load_best_model_at_end = True,
    # auto_find_batch_size = True,
    per_device_train_batch_size = batch_size,


    fp16=True,
    # bf16=True,
    # tf32=True,
    optim="adafactor",
    # optim="adamw_bnb",
    # fp16_full_eval=True,
    # jit_mode_eval=True,
    gradient_accumulation_steps = 32,
    gradient_checkpointing=True,

    logging_strategy="steps",
    logging_steps = logging_steps,

    seed = seed,

    run_name = model_type + f"_{model_name}_{dt_string}"
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    # eval_dataset = test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
    data_collator = lambda data : {
        'input_ids' : torch.stack([item['input_ids'] for item in data]),
        'attention_mask' : torch.stack([item['attention_mask'] for item in data]),
        'labels' : torch.stack([item['labels'] for item in data]),
    }
)

trainer.train()

test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

free_gpu()

model.eval()

pred = []
label = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, forced_bos_token_id=tokenizer.lang_code_to_id[tokenizer.tgt_lang])

        labels = batch['labels'].to(device)


        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        pred += decoded_preds
        label += decoded_labels



result = metric.compute(predictions=pred, references=label)
result = {"bleu": result["score"]}
print(result)


# model.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

MODE TYPE	:	mbart
SRC_LANG 	:	vi_VN
TGT_LANG 	:	en_XX
PREFIX   	:	None


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,1.234500
200,0.218600
300,0.209000
400,0.202700
500,0.197500


{'bleu': 38.08416361969523}


In [54]:
model.eval()

pred = []
label = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, forced_bos_token_id=tokenizer.lang_code_to_id[tokenizer.tgt_lang])

        labels = batch['labels'].to(device)


        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        pred += decoded_preds
        label += decoded_labels



result = metric.compute(predictions=pred, references=label)
result = {"bleu": result["score"]}
print(result)

{'bleu': 38.08416361969523}
